### Import Module

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation, LSTM, Dropout, TimeDistributed, Flatten
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.utils import class_weight
from sklearn.utils import class_weight
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from collections import Counter

### Data Preparation

In [2]:
training_load = pd.read_csv('training_1961_2016.csv', delimiter=',')
testing_2017_load = pd.read_csv('testing_2017.csv', delimiter=',')
testing_2018_load = pd.read_csv('testing_2018.csv', delimiter=',')
testing_2019_load = pd.read_csv('testing_2019.csv', delimiter=',')

In [3]:
training_load.head(5)

,FirstYear_index,FirstYear_playerID,FirstYear_yearID,FirstYear_teamID,FirstYear_lgID,FirstYear_stint,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,...,SixthYear_HBP,SixthYear_SH,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age
0,41261,aaronha01,1961,ML1,NL,1.0,155.0,603.0,115.0,197.0,...,1.0,0.0,8.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,32.0
1,41959,aaronha01,1962,ML1,NL,1.0,156.0,592.0,127.0,191.0,...,0.0,0.0,6.0,11.0,Hank,Aaron,Henry Louis,180.0,72.0,33.0
2,42719,aaronha01,1963,ML1,NL,1.0,161.0,631.0,121.0,201.0,...,1.0,0.0,5.0,21.0,Hank,Aaron,Henry Louis,180.0,72.0,34.0
3,43471,aaronha01,1964,ML1,NL,1.0,145.0,570.0,103.0,187.0,...,2.0,0.0,3.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,35.0
4,44225,aaronha01,1965,ML1,NL,1.0,150.0,570.0,109.0,181.0,...,2.0,0.0,6.0,13.0,Hank,Aaron,Henry Louis,180.0,72.0,36.0


In [4]:
var_name = ['index', 'playerID', 'teamID', 'lgID', 'stint','nameFirst', 'nameLast', 'nameGiven']

First = []
for i in range(len(var_name)):
    First.append(f"FirstYear_{var_name[i]}")

Second = []
for i in range(len(var_name)):
    Second.append(f"SecondYear_{var_name[i]}")

Third = []
for i in range(len(var_name)):
    Third.append(f"ThirdYear_{var_name[i]}")
    
Fourth = []
for i in range(len(var_name)):
    Fourth.append(f"FourthYear_{var_name[i]}")
    
Fifth = []
for i in range(len(var_name)):
    Fifth.append(f"FifthYear_{var_name[i]}")
    
FileColumnName = First + Second + Third + Fourth + Fifth

In [5]:
x_train = training_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(training_load), 5, 21))
y_train = training_load['SixthYear_HR'].values
x_test_2017 = testing_2017_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2017_load), 5, 21))
y_test_2017 = testing_2017_load['SixthYear_HR'].values
x_test_2018 = testing_2018_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2018_load), 5, 21))
y_test_2018 = testing_2018_load['SixthYear_HR'].values
x_test_2019 = testing_2019_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2019_load), 5, 21))
y_test_2019 = testing_2019_load['SixthYear_HR'].values

In [6]:
x_train_2018 = np.concatenate((x_train, x_test_2017, x_test_2018))
y_train_2018 = np.concatenate((y_train, y_test_2017, y_test_2018))

In [7]:
testing_2019 = testing_2019_load.drop(FileColumnName, axis=1).copy()

### Models

In [8]:
model_A = Sequential()
model_A.add(LSTM(128, input_shape=(5, 21), return_sequences=True))
model_A.add(LSTM(128, return_sequences=True))
model_A.add(Dropout(rate=0.5))
model_A.add(Flatten())
model_A.add(Dense(1024, activation="relu"))
model_A.add(Dropout(rate=0.5))
model_A.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_A.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_A.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 128)            76800     
                                                                 
 lstm_1 (LSTM)               (None, 5, 128)            131584    
                                                                 
 dropout (Dropout)           (None, 5, 128)            0         
                                                                 
 flatten (Flatten)           (None, 640)               0         
                                                                 
 dense (Dense)               (None, 1024)              656384    
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1

2024-03-08 15:21:04.769086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:04.770181: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:04.770853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [9]:
model_B = Sequential()
model_B.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_B.add(LSTM(16, return_sequences=True))
model_B.add(LSTM(8, return_sequences=True))
model_B.add(Dropout(rate=0.5))
model_B.add(Flatten())
model_B.add(Dense(512, activation="relu"))
model_B.add(Dropout(rate=0.5))
model_B.add(Dense(64, activation="relu"))
model_B.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_B.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_B.summary()

2024-03-08 15:21:05.015873: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:05.017060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:05.017651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 5, 32)             6912      
                                                                 
 lstm_3 (LSTM)               (None, 5, 16)             3136      
                                                                 
 lstm_4 (LSTM)               (None, 5, 8)              800       
                                                                 
 dropout_2 (Dropout)         (None, 5, 8)              0         
                                                                 
 flatten_1 (Flatten)         (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 512)               20992     
                                                                 
 dropout_3 (Dropout)         (None, 512)              

2024-03-08 15:21:05.284402: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:05.285589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:05.286283: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [10]:
model_C = Sequential()
model_C.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_C.add(LSTM(32, return_sequences=True))
model_C.add(BatchNormalization())
model_C.add(Flatten())
model_C.add(Dense(512, activation="relu"))
model_C.add(BatchNormalization())
model_C.add(Dense(64, activation="relu"))
model_C.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_C.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_C.summary()

2024-03-08 15:21:05.422676: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:05.423775: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:05.424443: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 5, 32)             6912      
                                                                 
 lstm_6 (LSTM)               (None, 5, 32)             8320      
                                                                 
 batch_normalization (BatchN  (None, 5, 32)            128       
 ormalization)                                                   
                                                                 
 flatten_2 (Flatten)         (None, 160)               0         
                                                                 
 dense_5 (Dense)             (None, 512)               82432     
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

2024-03-08 15:21:05.515854: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:05.516786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:05.517306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [11]:
model_D = Sequential()
model_D.add(LSTM(64, input_shape=(5, 21), return_sequences=True))
model_D.add(LSTM(64, return_sequences=True))
model_D.add(Dropout(rate=0.5))
model_D.add(TimeDistributed(Dense(10)))
model_D.add(Flatten())
model_D.add(Dense(512, activation="relu"))
model_D.add(Dropout(rate=0.5))
model_D.add(Dense(64, activation="relu"))
model_D.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_D.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_D.summary()

2024-03-08 15:21:05.673998: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:05.675099: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:05.675696: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 5, 64)             22016     
                                                                 
 lstm_8 (LSTM)               (None, 5, 64)             33024     
                                                                 
 dropout_4 (Dropout)         (None, 5, 64)             0         
                                                                 
 time_distributed (TimeDistr  (None, 5, 10)            650       
 ibuted)                                                         
                                                                 
 flatten_3 (Flatten)         (None, 50)                0         
                                                                 
 dense_9 (Dense)             (None, 512)               26112     
                                                      

2024-03-08 15:21:05.773376: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:05.773924: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:05.774454: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [12]:
model_E = Sequential()
model_E.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_E.add(LSTM(32, return_sequences=True))
model_E.add(Dropout(rate=0.5))
model_E.add(Flatten())
model_E.add(Dense(512, activation="relu"))
model_E.add(Dropout(rate=0.5))
model_E.add(Dense(64, activation="relu"))
model_E.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_E.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_E.summary()

2024-03-08 15:21:05.917783: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:05.918569: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:05.919505: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 5, 32)             6912      
                                                                 
 lstm_10 (LSTM)              (None, 5, 32)             8320      
                                                                 
 dropout_6 (Dropout)         (None, 5, 32)             0         
                                                                 
 flatten_4 (Flatten)         (None, 160)               0         
                                                                 
 dense_12 (Dense)            (None, 512)               82432     
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 64)               

2024-03-08 15:21:06.009046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 15:21:06.009608: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 15:21:06.010735: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

### Training

In [14]:
from sklearn.utils import class_weight

# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "A/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_A_stat = model_A.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000
79/79 [==============================] - ETA: 0s - loss: 137.0549 - accuracy: 0.0896
Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to A/saved-model-001-0.0524.hdf5
79/79 [==============================] - 2s 24ms/step - loss: 137.0549 - accuracy: 0.0896 - val_loss: 202.7393 - val_accuracy: 0.0524
Epoch 2/1000
79/79 [==============================] - ETA: 0s - loss: 131.0038 - accuracy: 0.0896
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 2s 24ms/step - loss: 131.0038 - accuracy: 0.0896 - val_loss: 198.8494 - val_accuracy: 0.0524
Epoch 3/1000
78/79 [============================>.] - ETA: 0s - loss: 141.0427 - accuracy: 0.0896
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 2s 23ms/step - loss: 141.5465 - accuracy: 0.0896 - val_loss: 408.2934 - val_accuracy: 0.0524
Epoch 4/1000
77/79 [============================>.] - ETA: 0s - loss: 150.8515 - accuracy: 0.0898
Epoch

In [15]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "B/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_B_stat = model_B.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 19:05:12.676662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:05:12.677511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:05:12.678317: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

73/79 [==========================>...] - ETA: 0s - loss: 782.5649 - accuracy: 0.0881

2024-03-08 19:05:15.206961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:05:15.207405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:05:15.207850: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to B/saved-model-001-0.0524.hdf5
79/79 [==============================] - 3s 13ms/step - loss: 765.7703 - accuracy: 0.0881 - val_loss: 331.5490 - val_accuracy: 0.0524
Epoch 2/1000
75/79 [===========================>..] - ETA: 0s - loss: 438.8449 - accuracy: 0.0899
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 433.0604 - accuracy: 0.0896 - val_loss: 400.0497 - val_accuracy: 0.0524
Epoch 3/1000
73/79 [==========================>...] - ETA: 0s - loss: 429.8208 - accuracy: 0.0898
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 418.4439 - accuracy: 0.0896 - val_loss: 386.9004 - val_accuracy: 0.0524
Epoch 4/1000
73/79 [==========================>...] - ETA: 0s - loss: 414.9442 - accuracy: 0.0896
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step 

In [16]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "C/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_C_stat = model_C.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 19:14:43.223935: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:14:43.224691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:14:43.225409: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

71/79 [=========================>....] - ETA: 0s - loss: 1183.2700 - accuracy: 0.0889

2024-03-08 19:14:45.148828: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:14:45.149346: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:14:45.150343: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to C/saved-model-001-0.0524.hdf5
79/79 [==============================] - 2s 11ms/step - loss: 1130.0713 - accuracy: 0.0884 - val_loss: 158.7647 - val_accuracy: 0.0524
Epoch 2/1000
79/79 [==============================] - ETA: 0s - loss: 570.1924 - accuracy: 0.0896
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/step - loss: 570.1924 - accuracy: 0.0896 - val_loss: 862.1815 - val_accuracy: 0.0524
Epoch 3/1000
76/79 [===========================>..] - ETA: 0s - loss: 397.9575 - accuracy: 0.0902
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 10ms/step - loss: 392.2317 - accuracy: 0.0896 - val_loss: 1133.7351 - val_accuracy: 0.0524
Epoch 4/1000
74/79 [===========================>..] - ETA: 0s - loss: 392.2444 - accuracy: 0.0892
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 7ms/st

In [17]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "D/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_D_stat = model_D.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 19:24:31.491369: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:24:31.492442: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:24:31.493416: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

75/79 [===========================>..] - ETA: 0s - loss: 541.3566 - accuracy: 0.0903

2024-03-08 19:24:33.700739: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:24:33.701750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:24:33.702273: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to D/saved-model-001-0.0524.hdf5
79/79 [==============================] - 3s 16ms/step - loss: 554.4691 - accuracy: 0.0895 - val_loss: 316.7348 - val_accuracy: 0.0524
Epoch 2/1000
78/79 [============================>.] - ETA: 0s - loss: 411.7870 - accuracy: 0.0896
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 12ms/step - loss: 411.1704 - accuracy: 0.0896 - val_loss: 368.0233 - val_accuracy: 0.0524
Epoch 3/1000
79/79 [==============================] - ETA: 0s - loss: 390.8068 - accuracy: 0.0896
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 12ms/step - loss: 390.8068 - accuracy: 0.0896 - val_loss: 298.4453 - val_accuracy: 0.0524
Epoch 4/1000
74/79 [===========================>..] - ETA: 0s - loss: 425.6450 - accuracy: 0.0906
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 12ms/st

In [18]:
from sklearn.utils import class_weight
# Correctly compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train_2018),
                                                  y=y_train_2018)

# Convert the class weights to a dictionary because Keras expects class weights in this format
class_weights_dict = dict(zip(np.unique(y_train_2018), class_weights))

class_weights_dict_int_keys = {int(key): value for key, value in class_weights_dict.items()}

# Detecting the highest class label
max_class_label = max(class_weights_dict_int_keys.keys())

# Ensuring continuity by replacing missing class with 0 as weight
adjusted_class_weights = {i: class_weights_dict_int_keys.get(i, 0) for i in range(max_class_label + 1)}

filepath = "E/saved-model-{epoch:03d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model_E_stat = model_E.fit(x_train_2018, y_train_2018, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2019, y_test_2019),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=adjusted_class_weights
                             )

Epoch 1/1000


2024-03-08 19:42:06.119731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:42:06.120282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:42:06.121074: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

77/79 [============================>.] - ETA: 0s - loss: 563.5993 - accuracy: 0.0884

2024-03-08 19:42:08.149425: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-08 19:42:08.149915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-08 19:42:08.150551: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_accuracy improved from -inf to 0.05236, saving model to E/saved-model-001-0.0524.hdf5
79/79 [==============================] - 2s 13ms/step - loss: 561.1547 - accuracy: 0.0890 - val_loss: 636.3383 - val_accuracy: 0.0524
Epoch 2/1000
78/79 [============================>.] - ETA: 0s - loss: 424.8665 - accuracy: 0.0895
Epoch 2: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 8ms/step - loss: 424.0502 - accuracy: 0.0896 - val_loss: 314.7780 - val_accuracy: 0.0524
Epoch 3/1000
74/79 [===========================>..] - ETA: 0s - loss: 384.5743 - accuracy: 0.0894
Epoch 3: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 9ms/step - loss: 394.9614 - accuracy: 0.0896 - val_loss: 288.1585 - val_accuracy: 0.0524
Epoch 4/1000
77/79 [============================>.] - ETA: 0s - loss: 378.3488 - accuracy: 0.0892
Epoch 4: val_accuracy did not improve from 0.05236
79/79 [==============================] - 1s 9ms/step 

### Analysis

In [19]:
AAA = load_model('2019/saved-model-851-0.0942-A.hdf5')
BBB = load_model('2019/saved-model-637-0.1257-B.hdf5')
CCC = load_model('2019/saved-model-203-0.0995-C.hdf5')
DDD = load_model('2019/saved-model-109-0.1099-D.hdf5')
EEE = load_model('2019/saved-model-714-0.1152-E.hdf5')

In [20]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = AAA.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 3ms/step
correct answer：0.0942
plus or minus 1：0.1675
plus or minus 3：0.3560
plus or minus 5：0.5183
plus or minus 10：0.7592


In [21]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.721404764664735
mean_squared_error: 77.85735240387437
rmse: 8.823681340793897


In [22]:
Counter(sorted(p))

Counter({7.0: 3,
         2.0: 2,
         12.0: 2,
         13.0: 2,
         4.0: 1,
         5.0: 1,
         6.0: 1,
         9.0: 1,
         10.0: 1,
         14.0: 1,
         20.0: 1,
         34.0: 1,
         35.0: 1})

In [23]:
Counter(sorted(n))

Counter({2.0: 3,
         3.0: 3,
         7.0: 3,
         12.0: 3,
         5.0: 2,
         6.0: 2,
         9.0: 2,
         13.0: 2,
         34.0: 2,
         1.0: 1,
         4.0: 1,
         10.0: 1,
         11.0: 1,
         14.0: 1,
         16.0: 1,
         17.0: 1,
         20.0: 1,
         25.0: 1,
         35.0: 1})

In [24]:
Counter(sorted(m))

Counter({12.0: 7,
         5.0: 6,
         2.0: 5,
         7.0: 5,
         9.0: 5,
         1.0: 4,
         3.0: 3,
         6.0: 3,
         0.0: 2,
         8.0: 2,
         13.0: 2,
         14.0: 2,
         15.0: 2,
         20.0: 2,
         23.0: 2,
         29.0: 2,
         34.0: 2,
         4.0: 1,
         10.0: 1,
         11.0: 1,
         16.0: 1,
         17.0: 1,
         19.0: 1,
         21.0: 1,
         22.0: 1,
         25.0: 1,
         27.0: 1,
         32.0: 1,
         35.0: 1})

In [25]:
Counter(sorted(q))

Counter({2.0: 4,
         1.0: 3,
         6.0: 3,
         3.0: 2,
         10.0: 2,
         12.0: 2,
         16.0: 2,
         17.0: 2,
         21.0: 2,
         23.0: 2,
         24.0: 2,
         33.0: 2,
         0.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         18.0: 1,
         19.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 1,
         34.0: 1,
         35.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         44.0: 1,
         48.0: 1,
         49.0: 1})

In [26]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [21, 25, 57, 64, 68, 69, 80, 83, 98, 114, 119, 125, 130, 136, 142, 153, 159, 177]
1: [4, 12, 21, 25, 29, 36, 37, 57, 64, 67, 68, 69, 74, 78, 80, 83, 91, 97, 98, 108, 114, 117, 118, 119, 125, 130, 136, 142, 153, 155, 159, 177]
3: [2, 4, 10, 12, 13, 21, 22, 25, 27, 28, 29, 36, 37, 42, 57, 60, 64, 67, 68, 69, 72, 74, 75, 78, 80, 82, 83, 88, 91, 92, 93, 94, 95, 96, 97, 98, 102, 104, 108, 109, 110, 111, 114, 117, 118, 119, 121, 123, 125, 127, 130, 136, 137, 142, 147, 148, 153, 154, 155, 157, 159, 160, 163, 164, 165, 167, 177, 183]
NA: [6, 16, 24, 34, 35, 41, 46, 47, 49, 50, 54, 61, 63, 65, 66, 71, 77, 89, 90, 99, 100, 105, 106, 107, 122, 126, 128, 139, 141, 144, 145, 151, 162, 166, 169, 170, 172, 173, 174, 176, 178, 180, 181, 187, 188, 190]


In [27]:
AA = testing_2019.copy()
AA['Prediction'] = np.round(predictions)
AA['Difference'] = AA.SixthYear_HR - AA.Prediction

In [28]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = BBB.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct：0.1257
plus or minus 1：0.2094
plus or minus 3：0.3822
plus or minus 5：0.5393
plus or minus 10：0.7696


In [29]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.392659028787263
mean_squared_error: 73.43778988846655
rmse: 8.569585164316097


In [30]:
Counter(sorted(p))

Counter({12.0: 4,
         3.0: 3,
         7.0: 2,
         15.0: 2,
         18.0: 2,
         5.0: 1,
         8.0: 1,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         13.0: 1,
         14.0: 1,
         16.0: 1,
         17.0: 1,
         27.0: 1,
         29.0: 1})

In [31]:
Counter(sorted(m))

Counter({12.0: 8,
         9.0: 7,
         11.0: 6,
         5.0: 5,
         2.0: 4,
         7.0: 4,
         13.0: 4,
         1.0: 3,
         3.0: 3,
         8.0: 3,
         14.0: 3,
         17.0: 3,
         20.0: 3,
         6.0: 2,
         10.0: 2,
         15.0: 2,
         18.0: 2,
         0.0: 1,
         4.0: 1,
         16.0: 1,
         21.0: 1,
         27.0: 1,
         29.0: 1,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [32]:
Counter(sorted(n))

Counter({12.0: 5,
         3.0: 3,
         9.0: 3,
         13.0: 3,
         2.0: 2,
         5.0: 2,
         7.0: 2,
         10.0: 2,
         11.0: 2,
         14.0: 2,
         15.0: 2,
         17.0: 2,
         18.0: 2,
         6.0: 1,
         8.0: 1,
         16.0: 1,
         20.0: 1,
         21.0: 1,
         27.0: 1,
         29.0: 1,
         32.0: 1})

In [33]:
Counter(sorted(q))

Counter({1.0: 3,
         2.0: 3,
         24.0: 3,
         33.0: 3,
         3.0: 2,
         4.0: 2,
         6.0: 2,
         21.0: 2,
         31.0: 2,
         34.0: 2,
         35.0: 2,
         0.0: 1,
         12.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 1,
         19.0: 1,
         23.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         41.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [34]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [13, 20, 21, 25, 29, 36, 57, 59, 72, 74, 78, 91, 97, 108, 110, 113, 121, 123, 131, 146, 153, 158, 165, 177]
1: [9, 12, 13, 20, 21, 25, 29, 36, 44, 57, 59, 60, 67, 69, 72, 74, 78, 80, 91, 97, 101, 104, 108, 110, 111, 113, 119, 121, 123, 124, 125, 130, 131, 137, 146, 153, 158, 165, 177, 185]
3: [2, 4, 7, 9, 11, 12, 13, 20, 21, 22, 23, 25, 27, 29, 36, 42, 44, 48, 55, 56, 57, 59, 60, 64, 67, 68, 69, 72, 73, 74, 75, 78, 80, 83, 86, 91, 95, 96, 97, 101, 102, 103, 104, 108, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 124, 125, 130, 131, 137, 142, 146, 147, 148, 153, 157, 158, 159, 160, 165, 167, 177, 184, 185]
NA: [6, 15, 24, 26, 34, 35, 38, 46, 47, 49, 54, 61, 63, 65, 66, 71, 79, 89, 99, 100, 105, 106, 107, 126, 128, 129, 141, 144, 145, 155, 156, 162, 166, 169, 170, 172, 173, 174, 178, 179, 180, 181, 182, 187]


In [35]:
BB = testing_2019.copy()
BB['Prediction'] = np.round(predictions)
BB['Difference'] = BB.SixthYear_HR - BB.Prediction

In [36]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = CCC.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.0995
plus or minus 1：0.2147
plus or minus 3：0.4031
plus or minus 5：0.5602
plus or minus 10：0.7958


In [37]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.138256144149141
mean_squared_error: 68.37441066441625
rmse: 8.2688820686001


In [38]:
Counter(sorted(p))

Counter({12.0: 5,
         2.0: 3,
         7.0: 2,
         8.0: 2,
         4.0: 1,
         6.0: 1,
         17.0: 1,
         20.0: 1,
         21.0: 1,
         23.0: 1,
         24.0: 1})

In [39]:
Counter(sorted(n))

Counter({12.0: 6,
         2.0: 5,
         3.0: 3,
         7.0: 2,
         8.0: 2,
         9.0: 2,
         10.0: 2,
         15.0: 2,
         17.0: 2,
         18.0: 2,
         1.0: 1,
         4.0: 1,
         6.0: 1,
         11.0: 1,
         13.0: 1,
         14.0: 1,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         23.0: 1,
         24.0: 1,
         27.0: 1,
         32.0: 1})

In [40]:
Counter(sorted(m))

Counter({12.0: 8,
         2.0: 6,
         5.0: 6,
         11.0: 5,
         1.0: 4,
         3.0: 4,
         9.0: 4,
         6.0: 3,
         7.0: 3,
         10.0: 3,
         14.0: 3,
         22.0: 3,
         0.0: 2,
         8.0: 2,
         15.0: 2,
         16.0: 2,
         17.0: 2,
         18.0: 2,
         20.0: 2,
         23.0: 2,
         4.0: 1,
         13.0: 1,
         21.0: 1,
         24.0: 1,
         27.0: 1,
         29.0: 1,
         31.0: 1,
         32.0: 1,
         34.0: 1})

In [41]:
Counter(sorted(q))

Counter({2.0: 4,
         1.0: 3,
         24.0: 3,
         33.0: 3,
         3.0: 2,
         35.0: 2,
         0.0: 1,
         4.0: 1,
         6.0: 1,
         9.0: 1,
         10.0: 1,
         12.0: 1,
         15.0: 1,
         16.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         23.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         31.0: 1,
         37.0: 1,
         39.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [42]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [7, 12, 21, 29, 43, 57, 60, 64, 67, 75, 80, 110, 119, 149, 159, 163, 165, 177, 184]
1: [7, 12, 20, 21, 25, 28, 29, 36, 42, 43, 57, 60, 64, 67, 69, 72, 74, 75, 78, 80, 91, 103, 110, 111, 113, 114, 119, 123, 125, 131, 142, 146, 148, 149, 153, 159, 163, 165, 168, 177, 184]
3: [2, 4, 5, 7, 9, 11, 12, 13, 18, 20, 21, 22, 23, 25, 28, 29, 32, 36, 39, 42, 43, 44, 57, 60, 64, 67, 68, 69, 72, 74, 75, 78, 79, 80, 88, 91, 92, 93, 94, 95, 96, 97, 101, 103, 104, 108, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 125, 131, 136, 142, 146, 147, 148, 149, 153, 154, 157, 159, 160, 163, 164, 165, 168, 175, 177, 184, 185]
NA: [6, 15, 24, 34, 35, 47, 49, 52, 54, 63, 65, 66, 71, 89, 99, 100, 106, 107, 122, 126, 128, 129, 139, 141, 145, 151, 166, 169, 170, 172, 173, 174, 178, 179, 180, 181, 182, 187, 190]


In [43]:
CC = testing_2019.copy()
CC['Prediction'] = np.round(predictions)
CC['Difference'] = CC.SixthYear_HR - CC.Prediction

In [44]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = DDD.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus 1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1099
plus or minus 1：0.2147
plus or minus 3：0.3979
plus or minus 5：0.5079
plus or minus 10：0.7435


In [45]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.972677904897959
mean_squared_error: 90.18605265621069
rmse: 9.496633753926213


In [46]:
Counter(sorted(p))

Counter({2.0: 5,
         9.0: 3,
         6.0: 2,
         7.0: 2,
         12.0: 2,
         13.0: 2,
         5.0: 1,
         8.0: 1,
         18.0: 1,
         32.0: 1,
         35.0: 1})

In [47]:
Counter(sorted(n))

Counter({2.0: 6,
         7.0: 5,
         1.0: 4,
         3.0: 3,
         9.0: 3,
         12.0: 3,
         6.0: 2,
         13.0: 2,
         29.0: 2,
         4.0: 1,
         5.0: 1,
         8.0: 1,
         14.0: 1,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         22.0: 1,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [48]:
Counter(sorted(m))

Counter({12.0: 9,
         2.0: 6,
         5.0: 6,
         1.0: 5,
         3.0: 5,
         7.0: 5,
         9.0: 4,
         0.0: 3,
         6.0: 3,
         23.0: 3,
         8.0: 2,
         10.0: 2,
         13.0: 2,
         15.0: 2,
         20.0: 2,
         22.0: 2,
         29.0: 2,
         4.0: 1,
         11.0: 1,
         14.0: 1,
         16.0: 1,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         21.0: 1,
         24.0: 1,
         25.0: 1,
         32.0: 1,
         34.0: 1,
         35.0: 1})

In [49]:
Counter(sorted(q))

Counter({2.0: 4,
         6.0: 3,
         23.0: 3,
         24.0: 3,
         33.0: 3,
         1.0: 2,
         3.0: 2,
         17.0: 2,
         19.0: 2,
         21.0: 2,
         31.0: 2,
         34.0: 2,
         35.0: 2,
         8.0: 1,
         10.0: 1,
         12.0: 1,
         14.0: 1,
         15.0: 1,
         16.0: 1,
         18.0: 1,
         22.0: 1,
         26.0: 1,
         27.0: 1,
         28.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         44.0: 1,
         48.0: 1,
         49.0: 1})

In [50]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [21, 22, 25, 55, 67, 83, 90, 94, 102, 111, 114, 118, 119, 130, 131, 136, 142, 147, 148, 171, 184]
1: [4, 10, 13, 18, 21, 22, 25, 42, 53, 55, 57, 59, 60, 67, 74, 78, 83, 88, 90, 91, 93, 94, 98, 102, 111, 114, 118, 119, 130, 131, 136, 142, 147, 148, 153, 157, 159, 167, 171, 177, 184]
3: [1, 2, 4, 5, 10, 13, 18, 21, 22, 25, 29, 32, 36, 37, 42, 45, 48, 53, 55, 57, 59, 60, 67, 69, 72, 74, 75, 77, 78, 80, 82, 83, 84, 88, 90, 91, 93, 94, 95, 96, 98, 102, 104, 108, 110, 111, 114, 118, 119, 121, 125, 127, 130, 131, 134, 136, 137, 140, 142, 146, 147, 148, 149, 150, 152, 153, 157, 159, 160, 163, 167, 168, 171, 175, 177, 184]
NA: [6, 15, 16, 24, 31, 34, 41, 46, 47, 50, 52, 54, 58, 61, 63, 65, 66, 71, 79, 86, 89, 99, 100, 105, 106, 107, 115, 126, 128, 129, 139, 141, 144, 145, 151, 155, 162, 166, 169, 170, 172, 173, 174, 176, 180, 181, 182, 187, 188]


In [51]:
DD = testing_2019.copy()
DD['Prediction'] = np.round(predictions)
DD['Difference'] = DD.SixthYear_HR - DD.Prediction

In [52]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = EEE.predict(x_test_2019)
for i in range(len(y_test_2019)):
    k = np.round(predictions)[i] - y_test_2019[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2019[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2019[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2019[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2019[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2019[i])
                        p1.append(i)
    else:
        q.append(y_test_2019[i])
        q1.append(i)
print("correct answer：%.4f" % (t/len(y_test_2019)))
print("plus or minus1：%.4f" % (a/len(y_test_2019)))
print("plus or minus 3：%.4f" % (b/len(y_test_2019)))
print("plus or minus 5：%.4f" % (c/len(y_test_2019)))
print("plus or minus 10：%.4f" % (d/len(y_test_2019)))

6/6 [==============================] - 0s 2ms/step
correct answer：0.1152
plus or minus1：0.2147
plus or minus 3：0.3822
plus or minus 5：0.5183
plus or minus 10：0.8010


In [53]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2019))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2019))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2019)))

mean_absolute_error: 6.57148983341237
mean_squared_error: 79.37487857303284
rmse: 8.909258025954397


In [54]:
Counter(sorted(p))

Counter({12.0: 4,
         3.0: 2,
         7.0: 2,
         11.0: 2,
         16.0: 2,
         2.0: 1,
         4.0: 1,
         5.0: 1,
         6.0: 1,
         10.0: 1,
         13.0: 1,
         15.0: 1,
         21.0: 1,
         23.0: 1,
         29.0: 1})

In [55]:
Counter(sorted(n))

Counter({12.0: 7,
         2.0: 4,
         9.0: 3,
         3.0: 2,
         5.0: 2,
         6.0: 2,
         7.0: 2,
         8.0: 2,
         10.0: 2,
         11.0: 2,
         13.0: 2,
         15.0: 2,
         16.0: 2,
         0.0: 1,
         1.0: 1,
         4.0: 1,
         17.0: 1,
         21.0: 1,
         23.0: 1,
         29.0: 1})

In [56]:
Counter(sorted(m))

Counter({12.0: 10,
         2.0: 6,
         5.0: 5,
         9.0: 5,
         1.0: 4,
         7.0: 4,
         0.0: 3,
         3.0: 3,
         6.0: 3,
         11.0: 3,
         13.0: 3,
         15.0: 3,
         17.0: 3,
         8.0: 2,
         10.0: 2,
         14.0: 2,
         16.0: 2,
         18.0: 2,
         23.0: 2,
         4.0: 1,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         27.0: 1,
         29.0: 1})

In [57]:
Counter(sorted(q))

Counter({2.0: 5,
         24.0: 3,
         33.0: 3,
         1.0: 2,
         3.0: 2,
         21.0: 2,
         26.0: 2,
         31.0: 2,
         34.0: 2,
         35.0: 2,
         16.0: 1,
         17.0: 1,
         23.0: 1,
         27.0: 1,
         28.0: 1,
         37.0: 1,
         38.0: 1,
         39.0: 1,
         41.0: 1,
         44.0: 1,
         45.0: 1,
         48.0: 1,
         49.0: 1})

In [58]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [7, 10, 21, 23, 36, 57, 60, 67, 74, 77, 91, 97, 108, 113, 118, 121, 130, 152, 159, 165, 177, 183]
1: [7, 9, 10, 12, 21, 23, 25, 29, 36, 42, 50, 55, 57, 60, 67, 69, 74, 75, 77, 80, 91, 95, 97, 108, 110, 113, 114, 118, 119, 121, 130, 142, 146, 148, 152, 159, 165, 175, 177, 183, 184]
3: [2, 4, 7, 9, 10, 12, 18, 20, 21, 22, 23, 25, 29, 33, 36, 42, 48, 50, 55, 57, 60, 67, 69, 72, 73, 74, 75, 77, 78, 80, 82, 87, 88, 90, 91, 93, 94, 95, 96, 97, 98, 102, 104, 108, 110, 113, 114, 118, 119, 121, 130, 131, 137, 140, 142, 146, 148, 152, 153, 154, 157, 158, 159, 160, 164, 165, 167, 171, 175, 177, 183, 184, 185]
NA: [6, 15, 19, 24, 34, 38, 47, 54, 61, 63, 65, 66, 71, 79, 81, 89, 99, 100, 106, 107, 126, 128, 129, 141, 144, 151, 155, 162, 166, 169, 170, 172, 173, 174, 179, 180, 182, 187]


In [59]:
EE = testing_2019.copy()
EE['Prediction'] = np.round(predictions)
EE['Difference'] = EE.SixthYear_HR - EE.Prediction

In [60]:
BB[(BB.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [61]:
DD[(DD.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [62]:
EE[(EE.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [63]:
AA[(AA.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [64]:
CC[(CC.Prediction > 39)]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference
